In [2]:
from sklearn import datasets
import matplotlib.pyplot as plt
from itertools import groupby
import numpy as np

iris = datasets.load_iris()

In [3]:
X = iris.data
y = iris.target

colors = { 0: "r", 1: "b", 2: "g" }

for key, grp in groupby(zip(X,y), lambda (_, j): j):
    subX = np.array([ e for (e, _) in grp ])
    plt.scatter(subX[:, 0], subX[:, 1], c=colors[key])
    
plt.show()

In [57]:
import math
# My own logistic regression with gradient descent

# the sigmoid function (also known as logistic)
def sigmoid(z):
    return 1.0 /(1.0 + math.exp(-z))



X_ = np.matrix([ np.concatenate(([1], row)) for row in X ])
y_ = np.matrix([1 if i == 0 else 0 for i in y]).transpose()

np.matrix(y).transpose()

sigmoidV = np.vectorize(sigmoid)
logV = np.vectorize(math.log)

# define the error function without regularization param
def J(X, y, theta):
    (m, _) = X.shape
    return (1.0/m) * (-y.transpose() * (logV(sigmoidV(X * theta))) \
                      - (1.0 - y).transpose() * (logV(1.0 - sigmoidV(X * theta))));

def grad(X, y, theta):
    (m, _) = X.shape
    return (1.0/m) * ((sigmoidV(X * theta) - y).transpose() * X).transpose();
    
errorthrs = 0.001


def evaluate(currentError, currentIters, theta, maxIters, iterErrList):
    if (currentError is not None and currentError[0,0] <= errorthrs) or currentIters >= maxIters:
        return iterErrList, currentError, theta
    
    newTheta = theta - 0.003 * grad(X_, y_, theta)
    newErr = J(X_, y_, newTheta)
    iterErrList.append(newErr[0,0])
    return evaluate(newErr, currentIters + 1, newTheta, maxIters, iterErrList)
    
def predict(trainingExamples, theta):
    return sigmoidV(trainingExamples * theta) >= 0.5
    

# add the bias unit to X
def error(X, y, theta):
    return np.mean(predict(X, theta) == y)

    
iterErrList, currentError, theta = evaluate(None, 0, np.matrix([[0],[0],[0],[0],[0]]), 500, [])

plt.plot(range(len(iterErrList)), iterErrList)
plt.show()

print predict(X_, theta) == y_
#print grad(X_, y_, [[0],[0],[0],[0],[0]])

[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 

In [ ]:
# Logistic regression with the library
from sklearn.linear_model import LogisticRegression
from sklearn import svm

clr = LogisticRegression()
clr.fit(X, y)
print clr.score(X, y) 

# SVM- Seems to fit better the data
clf = svm.SVC(kernel='linear', C=1).fit(X, y)
print clf.score(X, y) 